<img class="verticalhorizontal" src="../images/eossr_logo.png" alt="The eOSSR library" style="height:50px" vspace=20px /> <img class="verticalhorizontal" src="https://escape-ossr.gitlab.io/ossr-pages/img/Services_ESCAPE_OSSR-01.png
" alt="The ESCAPE OSSR" style="height:80px" vspace=0px />


# Validate and convert your metadata

This notebook will help you validate your metadata for an upload to the ESCAPE OSSR.

To do so, upload your codemeta metadata, either using an URL pointing to the `codemeta.json` file, uploading a `codemeta.json` file or copying the metadata in the text box below.

Note that you can generate your ESCAPE codemeta file using the online generator:
https://escape2020.pages.in2p3.fr/wp3/codemeta-generator/

In [ ]:
import ipywidgets as widgets
from eossr.metadata import codemeta
from eossr.metadata import codemeta2zenodo
from eossr.api import Record
import json

In [ ]:
text = widgets.Textarea(description= "codemeta:\n\n",
                        layout=widgets.Layout(height="500px", width="auto"))



## FROM URL

from urllib import request

def read_from_url(url):
    f = request.urlopen(url)
    meta = json.load(f)
    return meta

url = widgets.Text(description="URL:",
                   layout=widgets.Layout(width='auto'))


load_button = widgets.Button(description="Load")

def load(b):
    try:
        text.value = str(json.dumps(read_from_url(url.value), indent=4))
    except:
        text.value = "Could not read from URL"

        
load_button.on_click(load)
 
    
## FROM ZENODO RECORD

def read_from_record_id(record_id):
    rec = Record.from_id(record_id)
    meta = rec.get_codemeta()
    return meta

record_id = widgets.IntText(description="Record ID:",
                            layout=widgets.Layout(width='300px')
                           )

load_record_button = widgets.Button(description="Load")

def load_record(b):
    try:
        text.value = str(json.dumps(read_from_record_id(record_id.value), indent=4))
    except:
        text.value = f"Could not get codemeta from record {record_id.value}"

load_record_button.on_click(load_record)

## FROM UPLOAD

def read_from_upload(upload):   
    m = upload.value[0]['content']
    return json.loads(bytes(m))

upload = widgets.FileUpload(
    accept='.json',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    readonly=True
)


validate_button = widgets.Button(description="Validate !")

output = widgets.Textarea("", layout=widgets.Layout(height="200px", width="auto"))


def validate(b):
    try:
        metadata = json.loads(text.value)
        codemeta_handler = codemeta.Codemeta(metadata)
        output.value = f"Missing {codemeta_handler.missing_keys('required').size} required keys: " 
        output.value += f"{codemeta_handler.missing_keys('required')}\n"
        output.value += f"Missing {codemeta_handler.missing_keys('recommended').size} recommended keys: "
        output.value += f"{codemeta_handler.missing_keys('recommended')}"
                    
    except json.JSONDecodeError:
        output.value = "Invalid JSON input"
    except Exception as e:
        output.value = f"Error `{e}`, this can't be processed."
        
validate_button.on_click(validate)

load_label = widgets.Label("")
  
def load_upload(b):
    load_label.value = "Loaded"
    text.value = str(json.dumps(read_from_upload(upload), indent=4))

upload.observe(load_upload, names='value')

    
convert_button = widgets.Button(description="Convert to .zenodo.json",
                                layout=widgets.Layout(width="200px")
                               )
escape2020_checkbox = widgets.Checkbox(
    value=True,
    description='Add ESCAPE metadata',
    disabled=False,
    indent=False
)

zenodo_meta_text = widgets.Textarea(layout=widgets.Layout(width='auto', height='300px'))

def convert(convert_button):
    codemeta_dict = json.loads(text.value)
    zenodo_meta = codemeta2zenodo.converter(codemeta_dict, escape2020_checkbox.value)
    zenodo_meta_text.value = str(json.dumps(zenodo_meta, indent=4))

convert_button.on_click(convert)


In [ ]:
print("Load codemeta from a \033[95mjson file\n")
display(upload)
display(load_label)
print("\n")


blue='\033[94m'
print(f"Load codemeta from a \033[94mZenodo record ID\n")
display(record_id)
display(load_record_button)
print("\n")

print("Load codemeta from an \033[93mURL\n")
display(url)
display(load_button)
print("\n")

display(text, validate_button, output)
print("\n")

display(widgets.HBox([convert_button, escape2020_checkbox]))
display(zenodo_meta_text)